## This assignment may be worked individually or in pairs. 
## Enter your name/names here:
    

In [40]:
#names here
#name1: Pengdi Xia
#eid1: px353
#name2: Peijie Yang
#eid2: py2554


# Assignment 1: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1151` instances and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

0) The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.

1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

8-15) contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the 
number of lesions with the diameter of the ROI to compensate different image sizes. 

16) The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patient's condition. This feature is also normalized with the diameter of the ROI.

17) The diameter of the optic disc. 

18) The binary result of the AM/FM-based classification.

19) Class label. 1 = contains signs of Diabetic Retinopathy (Accumulative label for the Messidor classes 1, 2, 3), 0 = no signs of Diabetic Retinopathy.


A few function prototypes are already given to you, please don't change those. You can add additional helper functions for your convenience. *Suggestion:* The dataset is substantially big, for the purpose of easy debugging work with a subset of the data and test your decision tree implementation on that.

#### Implementation: 
A few function prototypes are already given to you, please don't change those. You can add additional helper functions for your convenience. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 buckets. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5.
- For binary features [0/1] the threshold value will be 1. All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [41]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import log
from random import shuffle

In [42]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    def __init__(self, label, features):
        self.label = label # the classification label of this data point
        self.features = features

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [43]:
def get_data(filename):
    data = []
#     your code goes here
    data_set = pd.read_csv(filename,header=None)
    for x in range(len(data_set)):
        datapoint = DataPoint((data_set.iloc[x][len(data_set.columns)-1]),
                              (data_set.iloc[x][:-1]))
        data.append(datapoint)
    return data

In [44]:
data = get_data('messidor_features.txt')

In [45]:
 class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self):    # for debugging purposes
        if self.is_leaf:
            print ('Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree()
            self.right_child.printTree()

Q2. Implement the function `make_prediction` that takes the decision tree root and a `DataPoint` instance and returns the prediction label.

In [46]:
def make_prediction(tree_root, data_point):
#     your code goes here
    while not tree_root.is_leaf:
        if data_point.features[tree_root.feature_idx] < tree_root.thresh_val:
            tree_root = tree_root.left_child
        else: 
            tree_root = tree_root.right_child
    return tree_root.prediction

Q3. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [47]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []
#     your code goes here
    for x in range(len(data)):
        if(data[x].features[feature_idx] < threshold):
            left_split.append(data[x])
        else: 
            right_split.append(data[x]) 
    return (left_split, right_split)

Q4. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [48]:
import math
def calc_entropy(data):
    entropy = 0.0
#     your code goes here
    total = len(data)
    count = 0
    ones = 0
    for x in range(total):
        if data[x].label == 0: 
            count = count + 1
    ones = total - count
    if(count == 0 or ones == 0):
        return 0
    entropy = -(count/total)*math.log2(count/total)-(ones/total)*math.log2(ones/total)
    return entropy

Q5. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [49]:
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = None
#     your code goes here
    parentEntropy = calc_entropy(data)
    if (feature_idx == 1 or feature_idx == 0 or feature_idx == 18):
        return 0, 1
    sortedData = sorted(data,key=lambda x: x.features[feature_idx])
    total = len(sortedData)
    zerosMore = 0
    onesMore = 0
    zerosLess = 0
    onesLess = 0
    gain = 0.0
    for x in range(total):
        if data[x].label == 0: 
            zerosMore = zerosMore + 1
    onesMore = total - zerosMore
    
    parentEntropy = calc_entropy(data)
    
    for x in range(total-1):
        average = (1/2)*(sortedData[x].features[feature_idx] + sortedData[x+1].features[feature_idx])
        if sortedData[x].label == 1: 
            onesLess+=1
            onesMore-=1
            if(onesMore != 0 and zerosMore != 0 and zerosLess != 0 ):
                gain = thresholdHelper(zerosMore, zerosLess, onesMore, onesLess, parentEntropy)
        else:
            zerosLess+=1
            zerosMore-=1
            if(onesMore != 0 and zerosMore != 0 and onesLess != 0 ):
                gain = thresholdHelper(zerosMore, zerosLess, onesMore, onesLess, parentEntropy)
        if (gain>=best_info_gain):
            best_info_gain = gain
            best_thresh = average
    return (best_info_gain, best_thresh)

In [50]:
def thresholdHelper(zerosMore, zerosLess, onesMore, onesLess, parentEntropy):
    lessEntropy = -(zerosLess/(zerosLess + onesLess))*math.log2(
        zerosLess/(zerosLess + onesLess)) - (onesLess/(zerosLess + onesLess))*math.log2(onesLess/(zerosLess + onesLess))
    moreEntropy = -(zerosMore/(zerosMore + onesMore))*math.log2(
        zerosMore/(zerosMore + onesMore)) -(onesMore/(zerosMore + onesMore))*math.log2(onesMore/(zerosMore + onesMore))
    gain = parentEntropy - ((zerosMore+onesMore)/(zerosMore+onesMore+zerosLess+onesLess))*moreEntropy-((zerosLess+onesLess)/(zerosMore+onesMore+zerosLess+onesLess))*lessEntropy
    return gain

Q6. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [51]:
def identify_best_split(data):
    if len(data) < 2:
        return (None, None)
    best_feature = None
    best_thresh = None
#     your code goes here
    best_gain = 0
    for x in range(len(data[0].features)):
        gain, thresh = calc_best_threshold(data, x)
        if gain > best_gain:
            best_gain = gain
            best_thresh = thresh
            best_feature = x
    return (best_feature, best_thresh)

Q7. Implement the function `createLeafNode` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node.

In [52]:
def createLeafNode(data):
#     your code goes here
    count = 0
    pred = 1
    for x in range(len(data)):
        if data[x].label == 0:
            count += 1
    if count > len(data)/2:
        pred = 0
    leafnode = TreeNode()
    leafnode.prediction = pred
    leafnode.is_leaf = True
    return leafnode

Q8. Implement the `createDecisionTree` function. `max_levels` denotes the maximum height of the tree (for example if `max_levels = 1` then the decision tree will only contain the leaf node at the root. [Hint: this is where the recursion happens.]

In [53]:
def createDecisionTree(data, max_levels):
#     your code goes here
# base case
    if max_levels == 1:
        return createLeafNode(data)
    else:
        best_feature, best_thresh = identify_best_split(data)
        if best_feature == None:
            return createLeafNode(data)
        node = TreeNode()
        left, right = split_dataset(data, best_feature, best_thresh)
        node.feature_idx = best_feature
        node.thresh_val = best_thresh
        node.is_leaf = False
        node.left_child = createDecisionTree(left, max_levels-1)
        node.right_child = createDecisionTree(right, max_levels-1)
    return node

Q9. Given a test set, the function `calcAccuracy` returns the accuracy of the classifier. You'll use the `makePrediction` function for this.

In [54]:
def calcAccuracy(tree_root, data):
#     your code goes here
    count=0
    for i in range (len(data)):
        if (make_prediction(tree_root,data[i]) == data[i].label):
            count += 1
    return count/len(data)

Q10. Keeping the `max_levels` parameter as 10, use 5-fold cross validation to measure the accuracy of the model. Print the recall and precision of the model. Also display the confusion matrix.

In [55]:
# edit the code here - this is just a sample to get you started
import time

d = get_data("messidor_features.txt")
split = 4
size = int (len(d)/4)
total = 0
for x in range(split):
# partition data into train_set and test_set
    train_set = d[:x*size] + d[(x+1)*size:]
    test_set = d[x*size:(x+1)*size]

    print ('Training set size:', len(train_set))
    print ('Test set size    :', len(test_set))

    # create the decision tree
    start = time.time()
    tree = createDecisionTree(train_set, 10)
    end = time.time()
    print ('Time taken:', end - start)

    # calculate the accuracy of the tree
    accuracy = calcAccuracy(tree, test_set)
    total = total + accuracy
    print ('The accuracy on the test set is ', str(accuracy * 100.0))
    #t.printTree()
print('The average accuracy on the test set is ', str((total/split)*100.0))

Training set size: 864
Test set size    : 287
Time taken: 4.33984112739563
The accuracy on the test set is  63.76306620209059
Training set size: 864
Test set size    : 287
Time taken: 4.218469858169556
The accuracy on the test set is  64.80836236933798
Training set size: 864
Test set size    : 287
Time taken: 4.237107038497925
The accuracy on the test set is  65.85365853658537
Training set size: 864
Test set size    : 287
Time taken: 4.377819061279297
The accuracy on the test set is  63.76306620209059
The average accuracy on the test set is  64.54703832752612


### To explore further:
1) Implement a pruning algorithm on the decision tree (either chi-squared, reduced error pruning, or model selection using a validation set/validation error) and see if that improves the generalization error of the decision tree (using 5-fold CV).

2) Use scikit-learn's decision tree to classify the data. Compare those results to your own. The documentation for scikit-learn's decision tree is very helpful and can be found [here](http://scikit-learn.org/stable/modules/tree.html).